In [1]:
import os
from Bio import SeqIO

In [2]:
def checkSlash(directory):
    if directory[-1] != '/':
        directory = directory + '/'
    return directory

# Edit prokka annotations for input into Tami's matlab pipeline

In [3]:
#prokkaDir = '09_prokka/'
#outDir = '09_prokka_edit/'

#Editing files for subgroup-assemblies
#DONE: subA, subB, subC, subD, subG, subJ
targetSubgroup = 'subE'

contigDir = '14_subgroup_contigs/'
prokkaDir = '14_subgroup_prokka/'
outDir = '14_subgroup_prokka_edit/'
prokkaDir = checkSlash(prokkaDir)
outDir = checkSlash(outDir)

In [4]:
#Function to edit an annotated file from prokka into separate *.gb files for each contig
#INPUTS  inContig: the reference contig file with headers that need to match the prokka results
#        inProkka: the input Prokka *.gbk file which needs to be parsed into separate contig files
#        sharedDelim: a text string prior to which is a unique identifier for the contig,
#                     e.g. use 'length' to produce NODE_1_ as a unique idenfier from a SPAdes assembly
#OUTPUT  outDir: path to directory where individual *.gb files will be stored, one for each contig, with headers
#                that match the input contig file
def exportFiles(inContig, inProkka, sharedDelim, outDir):
    with open(inProkka, 'r') as f:
        for line in f:
            if 'LOCUS' in line:
                if 'outFile' in locals():
                    outFile.close()
                lineList = line.strip().split()
                nodeName = lineList[1].split(sharedDelim)[0]
                correctName = ''
                records = list(SeqIO.parse(inContig, 'fasta'))
                for r in records:
                    if nodeName in r.id:
                        correctName = r.id
                if correctName == '':
                    print('error')
                outName = outDir + correctName + '.gb'
                outFile = open(outName, 'w')
                fixString = line.strip()
                noDef = True
            else:
                if noDef:
                    if 'DEFINITION' in line:
                        if len(fixString) < 79:
                            for i in range(79-len(fixString)):
                                fixString += ' '
                        outFile.write(fixString + '\n')
                        outFile.write(line)
                        noDef = False
                    else:
                        fixString += ' ' + line.strip()
                else:
                    outFile.write(line)

In [ ]:
#Wrapper for my personal directories
#if os.path.exists(outDir):
#    os.system('rm -r' + outDir)
for subdirs, dirs, files in os.walk(prokkaDir):
    for d in dirs:
        if d == targetSubgroup:
            for f in os.listdir(prokkaDir + d):
                if f[-4:] == '.gbk':
                    #make new directory for edited annotations
                    sampOutDir = outDir + d + '/'
                    if not os.path.exists(sampOutDir):
                        os.makedirs(sampOutDir)
                    contigFile = contigDir + d + '_contigs.fna'
                    print(d)
                    exportFiles(contigFile, prokkaDir + d + '/' + f, 'length', sampOutDir)

subE


KeyboardInterrupt: 

In [ ]:
#zip -r myfiles.zip mydir

# Edit prokka annotations for BioPython

In [6]:
def prokka_biopython_edit(prokkaFile, outFile):
    prokkaData = []
    with open(prokkaFile, 'r') as f:
        for line in f:
            prokkaData.append(line)
    outF = open(outFile, 'w')
    skipNext = False
    for i, line in enumerate(prokkaData):
        if 'LOCUS' in line:
            line = line.strip().split()
            locus = line[0]
            rawNode = line[1]
            length = rawNode.split('_')[3]
            if rawNode[-len(length):] == length:
                node = rawNode[0:len(rawNode) - len(length)]
            else:
                node = rawNode
            if len(line) < 6:
                date = prokkaData[i+1].strip()
                skipNext = True
            else:
                date = line[5]
            outLine = locus + '       ' + node + ' ' + length + ' bp DNA BCT linear 07-APR-2017\n'
            outF.write(outLine)
        else:
            if skipNext:
                skipNext = False
                continue
            outF.write(line)
    outF.close()

In [7]:
#Wrapper for the function above
inDir = '14_subgroup_prokka/'
outDir = '14_subgroup_prokka_edit/'
#inDir = '09_prokka'
#outDir = '09_prokka_biopython'
inDir = checkSlash(inDir)
outDir = checkSlash(outDir)
if not os.path.exists(outDir):
    os.makedirs(outDir)
for subdirs, dirs, files in os.walk(inDir):
    for d in dirs:
        sampID = d
        prokkaFile = inDir + checkSlash(d) + d + '.gbk'
        outFile = outDir + d + '.gb'
        prokka_biopython_edit(prokkaFile, outFile)